- It looks, like utils, that will work with unknown json, should not be
  universal
- Explicit ones should be for objects, another for arrays and rest for
  primitives
- Though public api probably should be universal (ready to unknown value)


#### I feel requirement in deterministic json serialization

Though it even looks like I can use deterministic stringify to simply compare
strings (or even it is already deterministic for the same process)... NOPE! I
have requirement to be ready for different values (leafs). So the determinism is
needed me for branch comparison, etc.


In [10]:
/// Suppose some example object
const example = {
    data: [{
        id: "asdf",
        user: {
            id: "asdfjasdf",
            name: "demo",
            is_admin: false,
        },
    }],
    pagination: {
        page: 1,
        limit: 2,
    },
};


#### So... i can generate deterministic key?


In [11]:
import { configure } from "npm:safe-stable-stringify";

const stringify = configure({ strict: true });

stringify(example);


'{"data":[{"id":"asdf","user":{"id":"asdfjasdf","is_admin":false,"name":"demo"}}],"pagination":{"limit":2,"page":1}}'

#### Not exactly

This stringify produce serialization of whole object... though it may be used
for final values.

## How to detect final value? (primitive)

# How to get final values with their paths?

because it looked like values aggregation only has no much sense


# Because I want return `json` that will describe some `json` I need strict convention about result structure

## In other words I can parse result of parsing as input value without confusion.

1. Strictly defined outer-level result type
2. For example
   `Record<"string" | "boolean" | "number" | "null" | "ARRAY" | "OBJECT", "???">`
3. Primitive values (so string/boolean/number/null keys) will simply have value
   as it is

# NO!

1. Though outer-level should be strictly defined, it should contain `type` key,
   to prevent iterating over possibilities, because at one time the value can be
   only one type
2. And so `value` also should be preset that will contain... whole input (actual
   for primitives only, but for convenient will be present always)
3.


In [12]:
function lab(value: unknown) {
    const result = {} as {
        type: "null" | "string" | "number" | "boolean" | "ARRAY" | "OBJECT";
    };

    if (typeof value === "object") {
        if (Array.isArray(value)) {
            result.type = "ARRAY";
        } else if (value === null) {
            result.type = "null";
        } else {
            result.type = "OBJECT";
        }
    } else if (typeof value === "number") {
        result.type = "number";
    } else if (typeof value === "string") {
        result.type = "string";
    } else if (typeof value === "boolean") {
        result.type = "boolean";
    }

    return result;
}

console.log(
    lab(example),
    lab(null),
    lab("str"),
    lab(2.2),
    lab(false),
);


{ type: "OBJECT" } { type: "null" } { type: "string" } { type: "number" } { type: "boolean" }
